<a href="https://colab.research.google.com/github/peteray-dev/JP_task_oil_and_gas_exploration/blob/master/JP_task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('./Nat_Gas (1).csv')

In [ ]:
df.head()

,Dates,Prices
0,10/31/20,10.1
1,11/30/20,10.3
2,12/31/20,11.0
3,1/31/21,10.9
4,2/28/21,10.9


In [ ]:
# df['Dates'] = pd.to_datetime(df['Dates'], format='%m/%d/%y')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Dates   48 non-null     object 
 1   Prices  48 non-null     float64
dtypes: float64(1), object(1)
memory usage: 896.0+ bytes


In [ ]:
p = df[df['Dates'] == '10/31/20'].values[0][1]
p

10.1

In [ ]:
max_storage_volume = 1e6  # Max storage capacity in MMBtu
storage_cost_per_month = 100000  # Monthly storage cost
injection_cost_per_mmbtu = 10000  # Injection cost per million MMBtu
withdrawal_cost_per_mmbtu = 10000  # Withdrawal cost per million MMBtu
transport_cost_per_injection = 50000  # Transport cost per injection
transport_cost_per_withdrawal = 50000  # Transport cost per withdrawal

In [ ]:
def prices_gas_contract(
    injection_dates,
    withdrawal_dates,
    prices,
    injection_volumes,
    withdrawal_volumes,
    max_storage_volume,
    storage_cost_per_month,
    injection_cost_per_mmbtu,
    withdrawal_cost_per_mmbtu,
    transport_cost_per_injection,
    transport_cost_per_withdrawal
):
  total_value = 0
  storage_volume = 0

  for date, inj_volume in zip(injection_dates, injection_volumes):
    if storage_volume + inj_volume > max_storage_volume:
      print(f'Injection date {date} exceeds storage capacity.')

    purchase_price = df[df['Dates'] == date].values[0][1]
    purchase_cost = purchase_price * inj_volume

    injection_cost = injection_cost_per_mmbtu * (inj_volume/1e6)
    transport_cost = transport_cost_per_injection

    storage_volume += inj_volume
    total_value -= (purchase_cost + injection_cost + transport_cost)
    # print(f'value at inj {total_value}')

  for date, withdrawal_volume in zip(withdrawal_dates, withdrawal_volumes):
    if withdrawal_volume > storage_volume:
      print(f'Withdrawal date {date} exceeds storage volume. available volume {storage_volume}')

    # asuuming that the price on a particular day is the amount you will buy and also sell
      sell_price = df[df['Dates'] == date].values[0][1]
      sell_cost = sell_price * withdrawal_volume

      withdrawal_cost = withdrawal_cost_per_mmbtu * (withdrawal_volume/1e6)
      transport_cost = transport_cost_per_withdrawal

      storage_volume -= withdrawal_volume
      total_value += (sell_cost - withdrawal_cost - transport_cost)
      # print(f'value at withdrawal {total_value}')
  # injection_dates = pd.to_datetime(injection_dates)
  first_injection= pd.to_datetime(injection_dates[0])
  last_injection = pd.to_datetime(injection_dates[-1])
  first_withdrawal = pd.to_datetime(withdrawal_dates[0])
  last_withdrawal = pd.to_datetime(withdrawal_dates[-1])
  month_of_storage = (last_withdrawal.year - first_injection.year) * 12 + (last_withdrawal.month - first_injection.month)
  total_storage_cost = storage_cost_per_month * month_of_storage
  total_value -= total_storage_cost

  return total_value



In [ ]:
# Dates for injecting and withdrawing gas
injection_dates = ['10/31/20', '5/31/21']
withdrawal_dates = ['12/31/21', '9/30/22']

# Injection and withdrawal volumes (in MMBtu)
injection_volumes = [0.1e6, 0.5e6]
withdrawal_volumes = [0.2e6, 0.2e6]


# Other parameters
max_storage_volume = 1e6  # Max storage capacity in MMBtu
storage_cost_per_month = 100000  # Monthly storage cost
injection_cost_per_mmbtu = 10000  # Injection cost per million MMBtu
withdrawal_cost_per_mmbtu = 10000  # Withdrawal cost per million MMBtu
transport_cost_per_injection = 50000  # Transport cost per injection
transport_cost_per_withdrawal = 50000  # Transport cost per withdrawal


In [ ]:
contract_value = prices_gas_contract(
    injection_dates,
    withdrawal_dates,
    df,
    injection_volumes,
    withdrawal_volumes,
    max_storage_volume,
    storage_cost_per_month,
    injection_cost_per_mmbtu,
    withdrawal_cost_per_mmbtu,
    transport_cost_per_injection,
    transport_cost_per_withdrawal
)

print(f"The value of the gas contract is: ${contract_value:.2f}")

The value of the gas contract is: $-8336000.00


In [ ]:
from datetime import date
import math

def price_contract(in_dates, in_prices, out_dates, out_prices, rate, storage_cost_rate, total_vol, injection_withdrawal_cost_rate):
    volume = 0
    buy_cost = 0
    cash_in = 0
    last_date = min(min(in_dates), min(out_dates))

    # Ensure dates are in sequence
    all_dates = sorted(set(in_dates + out_dates))

    for i in range(len(all_dates)):
        # processing code for each date
        start_date = all_dates[i]

        if start_date in in_dates:
            # Inject on these dates and sum up cash flows
            if volume <= total_vol - rate:
                volume += rate

                # Cost to purchase gas
                buy_cost += rate * in_prices[in_dates.index(start_date)]
                # Injection cost
                injection_cost = rate * injection_withdrawal_cost_rate
                buy_cost += injection_cost
                print('Injected gas on %s at a price of %s'%(start_date, in_prices[in_dates.index(start_date)]))

            else:
                # We do not want to inject when rate is greater than total volume minus volume
                print('Injection is not possible on date %s as there is insufficient space in the storage facility'%start_date)
        elif start_date in out_dates:
            #Withdraw on these dates and sum cash flows
            if volume >= rate:
                volume -= rate
                cash_in += rate * out_prices[out_dates.index(start_date)]
                # Withdrawal cost
                withdrawal_cost = rate * injection_withdrawal_cost_rate
                cash_in -= withdrawal_cost
                print('Extracted gas on %s at a price of %s'%(start_date, out_prices[out_dates.index(start_date)]))
            else:
                # we cannot withdraw more gas than is actually stored
                print('Extraction is not possible on date %s as there is insufficient volume of gas stored'%start_date)

    store_cost = math.ceil((max(out_dates) - min(in_dates)).days // 30) * storage_cost_rate
    return cash_in - store_cost - buy_cost

# Example usage of price_contract()
in_dates = [date(2022, 1, 1), date(2022, 2, 1), date(2022, 2, 21), date(2022, 4, 1)] #injection dates
in_prices = [20, 21, 20.5, 22]#prices on the injection days
out_dates = [date(2022, 1, 27), date(2022, 2, 15), date(2022, 3, 20), date(2022, 6, 1)] # extraction dates
out_prices = [23, 19, 21, 25] # prices on the extraction days
rate = 100000  # rate of gas in cubic feet per day
storage_cost_rate = 10000  # total volume in cubic feet
injection_withdrawal_cost_rate = 0.0005  # $/cf
max_storage_volume = 500000 # maximum storage capacity of the storage facility
result = price_contract(in_dates, in_prices, out_dates, out_prices, rate, storage_cost_rate, max_storage_volume, injection_withdrawal_cost_rate)
print()
print(f"The value of the contract is: ${result}")





Injected gas on 2022-01-01 at a price of 20
Extracted gas on 2022-01-27 at a price of 23
Injected gas on 2022-02-01 at a price of 21
Extracted gas on 2022-02-15 at a price of 19
Injected gas on 2022-02-21 at a price of 20.5
Extracted gas on 2022-03-20 at a price of 21
Injected gas on 2022-04-01 at a price of 22
Extracted gas on 2022-06-01 at a price of 25

The value of the contract is: $399600.0
